## DCGANによる画像生成


In [4]:
import torch
from torch import nn, optim
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import TensorDataset, DataLoader, Dataset
import tqdm

In [6]:
img_data = ImageFolder("../data/oxford-102/",
                       transform = transforms.Compose([transforms.Resize(80),
                                                       transforms.CenterCrop(64),
                                                       transforms.ToTensor()]))
batch_size = 64
img_loader = DataLoader(img_data, batch_size = batch_size, shuffle = True)

In [7]:
nz = 100
ngf = 32

In [9]:
class Gnet(nn.Module):
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf * 8,
                               4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.reLU(inplace=True),
            nn.ConvTranspose2d(ngf * 8, ngf * 4,
                               4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(ngf * 2, ngf,
                               4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(ngf, 3,
                               4, 2, 1, bias=False),
            nn.Tanf()
        )
    def forward(self, x):
        out = self.main(x)
        return out

## 識別モデルの作成

In [10]:
ndf = 32

In [14]:
class DNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False)
        )
    def forward(self, x):
        out = self.main(x)
        return out.squeeze()

In [15]:
d = DNet().to("cuda:0")
g = GNet().to("cuda:0")

# ADAMのパラメータは元論文の提案値
opt_d = optim.Adam(d.parameters(),
                   lr = 0.0002, betas=(0.5, 0.999))
opt_g = optim.Adam(g.parameters(),
                   lr = 0.0002, betas=(0.5, 0.999))

ones = torch.ones(batch_size).to("cuda:0")
zeros = torch.zeros(batch_size).to("cuda:0")
loss_f = nn.BCEWITHLogitsLoss()

fixed_z = torch.randn(batch_size, nz, 1, 1).to("cuda:0")

RuntimeError: No CUDA GPUs are available